In [1]:
:opt no-lint

# $\lambda$-calculus evaluator

call-by-name evaluation

In [2]:
-- 변수 이름은 문자열 나타낸다
type Nm = String

-- 람다식 문법 구조
data Tm = Var Nm       -- x
        | Lam Nm Tm    -- (λx.e)
        | App Tm Tm    -- (e1 e2)
        deriving (Show, Eq)

In [3]:
-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x e) = "\\" ++ x ++ " -> " ++ ppTm e
ppTm (App e1 e2) = pp1 e1 ++ " " ++ pp2 e2
  where
  pp1 e@(Lam{}) = paren (ppTm e)
  pp1 e         = ppTm e
  pp2 e@(Var{}) = ppTm e
  pp2 e         = paren (ppTm e)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"
latex s = "$" ++ s ++ "$"

-- 람다식을 보기좋게 TeX 코드로 변환해주는 함수
texTm (Var x) = x
texTm (Lam x e) = "\\lambda " ++ x ++ "." ++ texTm e
texTm (App e1 e2) = tex1 e1 ++ "~" ++ tex2 e2
  where
  tex1 e@(Lam{}) = paren (texTm e)
  tex1 t         = texTm t
  tex2 s@(Var{}) = texTm s
  tex2 s         = paren (texTm s)

In [4]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

putStr $ ppTm idTm
putStr $ ppTm ttTm
putStr $ ppTm ffTm

\x -> x

\x -> \y -> x

\x -> \y -> y

In [5]:
import IHaskell.Display

html . latex $ texTm idTm
html . latex $ texTm ttTm
html . latex $ texTm ffTm
html . latex $ texTm (App (App (Var "x") (Var "y")) (Var "z"))
html . latex $ texTm (App (Var "x") (App (Var "y") (Var "z")))
html . latex $ texTm (App (App ffTm idTm) ttTm)
html . latex $ texTm (App ffTm (App idTm ttTm))

$\lambda x.x$

$\lambda x.\lambda y.x$

$\lambda x.\lambda y.y$

$x~y~z$

$x~(y~z)$

$(\lambda x.\lambda y.y)~(\lambda x.x)~(\lambda x.\lambda y.x)$

$(\lambda x.\lambda y.y)~((\lambda x.x)~(\lambda x.\lambda y.x))$

In [6]:
type Env = [ (Nm, Value) ]
type Value = Addr
type Heap = [(Addr, Thunk)]
type Thunk = (Env,Tm)
type Addr = Int

force :: Heap -> Value -> Heap
force h v =
  case e of Lam{} -> h
            _     -> let (h',v') = eval h env e
                      in update v (lookup' v' h') h'
  where
    (env,e) = lookup' v h

thunk :: Heap -> Value -> Thunk -> Heap
thunk h v th = (v,th):h

newAddr [] = 0
newAddr h = 1 + maximum (map fst h)

eval :: Heap -> Env -> Tm -> (Heap,Value)
eval h env (Var x) =
  case e1 of Lam{} -> (h,v)
             _     -> (force h v, v)
  where
    v = lookup' x env
    (env1,e1) = lookup' v h
eval h env e@(Lam{}) = (thunk h v (env,e), v)
  where v = newAddr h
eval h env (App e1 e2) =
  case e of
    Lam x eb -> eval ((a2,(env,e2)):h1) ((x,a2):env1) eb
    _        -> error (show e1 ++ " cannot force to Lam")
  where
    (h1,v1) = eval h env e1
    a2 = newAddr h1
    (env1,e) = lookup' v1 h1

lookup' k ps = case lookup k ps of
                 Nothing -> error (show k ++ " not defined")
                 Just v  -> v

update x v [] = []
update x v (p@(y,_):ps)
  | x == y    = (x,v) : ps
  | otherwise = p : update x v ps

In [7]:
import Data.List (intersperse)

texHeapValue :: (Heap,Value) -> String
texHeapValue (h,v) = "*"++show v++" \\\\ " ++ texHeap h
texHeap :: Heap -> String
texHeap h = "[\\,"
         ++ (concat . intersperse "\\\\,")
                [show a++"\\mapsto "++texThunk th | (a,th) <-h]
         ++ "\\,]"
texThunk :: Thunk -> String
texThunk (env,e) = "\\langle"++texEnv env++","++texTm e++"\\rangle"
texEnv :: Env -> String
texEnv env = "\\{"
          ++ (concat . intersperse ",")
                 [x++"\\mapsto "++show v | (x,v) <-env]
          ++ "\\}"

In [19]:
e1 = (Lam "f" $ Lam "z" $ App (Lam "x" $ Var "f" `App` Var "x") (Var "z"))
   `App`
     (App (Lam "v" $ Var "v") (Lam "w" $ Var "w"))

html . latex . texTm $ e1
html . latex . texHeapValue $ eval [] [] e1

$(\lambda f.\lambda z.(\lambda x.f~x)~z)~((\lambda v.v)~(\lambda w.w))$

$*2 \\ [\,2\mapsto \langle\{f\mapsto 1\},\lambda z.(\lambda x.f~x)~z\rangle\\,1\mapsto \langle\{\},(\lambda v.v)~(\lambda w.w)\rangle\\,0\mapsto \langle\{\},\lambda f.\lambda z.(\lambda x.f~x)~z\rangle\,]$

In [9]:
e2 = (Lam "x" $ Lam "y" (Var "y" `App` (Var "y" `App` Var "y")) `App` Var "x")
  `App` (Lam "z" $ Var "z")
html . latex . texTm $ e2
html . latex . texHeapValue $ eval [] [] e2

$(\lambda x.(\lambda y.y~(y~y))~x)~(\lambda z.z)$

$*4 \\ [\,5\mapsto \langle\{\},\lambda z.z\rangle\\,4\mapsto \langle\{\},\lambda z.z\rangle\\,3\mapsto \langle\{\},\lambda z.z\rangle\\,2\mapsto \langle\{x\mapsto 1\},\lambda y.y~(y~y)\rangle\\,1\mapsto \langle\{\},\lambda z.z\rangle\\,0\mapsto \langle\{\},\lambda x.(\lambda y.y~(y~y))~x\rangle\,]$

In [10]:
e3 = (Lam "x" $ Lam "y" (Var "y" `App` (Var "y" `App` Var "y")) `App` Var "x")
  `App` (Lam "z" $ Var "z") `App` (Lam "w" $ Var "w")
html . latex . texTm $ e3
html . latex . texHeapValue $ eval [] [] e3

$(\lambda x.(\lambda y.y~(y~y))~x)~(\lambda z.z)~(\lambda w.w)$

$*6 \\ [\,6\mapsto \langle\{\},\lambda w.w\rangle\\,5\mapsto \langle\{\},\lambda z.z\rangle\\,4\mapsto \langle\{\},\lambda z.z\rangle\\,3\mapsto \langle\{\},\lambda z.z\rangle\\,2\mapsto \langle\{x\mapsto 1\},\lambda y.y~(y~y)\rangle\\,1\mapsto \langle\{\},\lambda z.z\rangle\\,0\mapsto \langle\{\},\lambda x.(\lambda y.y~(y~y))~x\rangle\,]$

In [11]:
e5 = (Lam "x" $ Lam "y" $ Var "x") `App` (Lam "z" $ Var "z") `App` (App omega omega)
omega = (Lam "x" $ Var "x" `App` Var "x") `App` (Lam "x" $ Var "x" `App` Var "x")

html . latex . texTm $ e5
html . latex . texHeapValue $ eval [] [] e5

$(\lambda x.\lambda y.x)~(\lambda z.z)~((\lambda x.x~x)~(\lambda x.x~x)~((\lambda x.x~x)~(\lambda x.x~x)))$

$*1 \\ [\,3\mapsto \langle\{\},(\lambda x.x~x)~(\lambda x.x~x)~((\lambda x.x~x)~(\lambda x.x~x))\rangle\\,2\mapsto \langle\{x\mapsto 1\},\lambda y.x\rangle\\,1\mapsto \langle\{\},\lambda z.z\rangle\\,0\mapsto \langle\{\},\lambda x.\lambda y.x\rangle\,]$

In [12]:
-- html . latex . texTm $ omega
-- html . latex . texHeapValue $ eval [] [] omega